In [47]:
import fitz

pdf_path = "cmd.pdf"
output_file = "CMV_output.txt"

def annotation_extractor(pdf_path, output_file):
    doc = fitz.open(pdf_path)

    with open(output_file, "w") as f:
        for i in range(len(doc)):
            page = doc[i]
            annotations = page.annots()

            if annotations:
                f.write(f"Page {i + 1}:\n")
                for annotation in annotations:
                    rect = annotation.rect
                    if rect:
                        f.write(f"  - {page.get_textbox(rect)}\n")
                    else:
                        f.write("  - None\n")
                f.write("\n")
            else:
                f.write(f"Page {i + 1}: No annotations\n\n")

    doc.close()

# Call the function
annotation_extractor(pdf_path, output_file)


In [46]:
import fitz
import csv
import json
import xml.etree.ElementTree as ET

pdf_path = "cmd.pdf"

# Output file paths
output_csv = "cmv_output.csv"
output_json = "cmv_output.json"
output_xml = "cmv_output.xml"

def annotation_extractor(pdf_path, output_file):
    doc = fitz.open(pdf_path)
    annotations_data = []

    for i in range(len(doc)):
        page = doc[i]
        annotations = page.annots()

        if annotations:
            page_data = {
                "page": i + 1,
                "annotations": []
            }
            for annotation in annotations:
                rect = annotation.rect
                if rect:
                    annotation_data = {
                        "content": page.get_textbox(rect)
                    }
                else:
                    annotation_data = {
                        "content": "None"
                    }
                page_data["annotations"].append(annotation_data)
            annotations_data.append(page_data)

    doc.close()

    # Save as CSV
    with open(output_csv, "w", newline="") as csv_file:
        fieldnames = ["Page", "Content"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for page_data in annotations_data:
            for annotation in page_data["annotations"]:
                writer.writerow({
                    "Page": page_data["page"],
                    "Content": annotation["content"]
                })

    # Save as JSON
    with open(output_json, "w") as json_file:
        json.dump(annotations_data, json_file, indent=4)

    # Save as XML
    root = ET.Element("annotations")
    for page_data in annotations_data:
        page_element = ET.SubElement(root, "page", number=str(page_data["page"]))
        for annotation in page_data["annotations"]:
            annotation_element = ET.SubElement(page_element, "annotation")
            content_element = ET.SubElement(annotation_element, "content")
            content_element.text = annotation["content"]

    tree = ET.ElementTree(root)
    tree.write(output_xml)

# Call the function
annotation_extractor(pdf_path, output_file)
